### Clipping The netcdf files by shapefiles

In [ ]:
#Let's import some libraries that we need. 

import xarray as xr
import numpy as np
import regionmask 
import geopandas as gpd
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import os, sys
import warnings; warnings.filterwarnings(action='ignore')
%matplotlib inline

#Getting the currecnt working directory
pwd = os.getcwd()
os.chdir(pwd)
print(pwd)

In [ ]:
shapefile = "/Users/michaelrogo/software/cli_he_code/hacked/NUTS_RG_20M_2021_4326/NUTS_RG_20M_2021_4326.shp"
countries = gpd.read_file(shapefile, header='None')
querry = 3
countries = countries[countries['LEVL_CODE'] == querry]
#countries['NUTS_ID'].to_csv('file.csv', header=False)


with open('codes.csv', 'a') as file:
        if os.path.getsize('codes.csv') == 0:
            file.write('ID_NO,NUTS_ID\n')
            countries['NUTS_ID'].to_csv(file, header=False, index=True)
        else:
            pass

df = pd.read_csv("codes.csv")
print(len(df))
my_dict = dict(zip(df['ID_NO'], df['NUTS_ID']))
#my_dict

In [ ]:
#----------------------------------------
#  Let's plot the area presented by shape file
#----------------------------------------
#fig, ax = plt.subplots(figsize=(16, 10))
#countries.plot(ax=ax, column='NUTS_ID')
#countries.head()['NUTS_ID']
#.to_pandas().to_csv("nuts_id.csv")

## Comes the .nc file

In [ ]:
##----------------------------------------
#Read the Netcdf file 
#The following command is for data with single variables of the nc file masked to the shape file
#To reproduce the results, please create the respective paths manually on disk and not by the code!! 
#One can choose desired paths
#RESULTS
# We get a bunch of csv files for each region with values. We use the code in the next cell to process them into one 
#huge file. But even at this level, we can get some insights
##----------------------------------------
#print(pwd)
path = '/Users/michaelrogo/software/cli_he_code/hacked/' #path where we have the nc file

#dest = '/Users/michaelrogo/software/cli_he_code/hacked/max_temp/max_files' #path where we dump nc files (not used at the moment) 
#The code first identifies the coordinates of regions in the nc file and mask them with the shape file. 
#We then use the mask to extract the values of interest (looped over every region of interest). 

for filename in os.listdir(path):
    f = os.path.join(path,filename)
    if f.endswith('.nc'):
        year = f[-7:-3]
        
        #read nc file and assign lons and lats of interest
        d = xr.open_mfdataset(f, chunks = {'time': 10})
        d = d.assign_coords(longitude=(((d.longitude + 180) % 360) - 180)).sortby('longitude')
        #print("\n ---------------\nShowing NetCDF4 for {}.....\n ---------------\n".format(f), d)
        #CALCULATE MASK
        nuts = countries
        #names = list(nuts.NUTS_ID)
        #len(names)
        
        #mask these coordinates to each region. The length of the nuts region is 1514 (from the shape file) 
        #The line is long and broken by '\' symbol
        nuts_mask_poly = regionmask.Regions(name = 'nuts_mask', numbers = list(range(0,1514))\
        , names = list(nuts.NUTS_ID), abbrevs = list(nuts.NUTS_ID)\
        , outlines = list(nuts.geometry.values[i] for i in range(0,1514)))

        print("--------------------------------\n\
                Masking {}...\n--------------------------------\n\
                ".format(f),nuts_mask_poly)
        mask = nuts_mask_poly.mask(d.isel(time=0), lat_name='latitude', lon_name='longitude')
        
        #Here we read the IDs of each region in a loop to extract point data. As is clear, the loop tries
        #to skip missing values which are problematic when we have NaN within the values extracted (basically
        #empty NUTS regions)
        nuts_ids = []
        for i in my_dict:
            ID_REGION = i
            nuts_ids.append(ID_REGION)
            lat = mask.latitude.values
            lon = mask.longitude.values
            sel_mask = mask.where(mask == ID_REGION).values
            #print(sel_mask)
            a = []
            id_lon = lon[np.where(~np.all(np.isnan(sel_mask), axis=0))]
            id_lat = lat[np.where(~np.all(np.isnan(sel_mask), axis=1))]
            if len(id_lon) | len(id_lat) :
                #print(ID_REGION, "\t lon-->", id_lon)
                #print(ID_REGION, "\t lat-->", id_lat)
                print("\n")
                out_sel = d.sel(latitude = slice(id_lat[0], id_lat[-1]), longitude = slice(id_lon[0], id_lon[-1])).compute().where(mask == ID_REGION)
                #print(out_sel) 
                print("\n------Extracting Temp Data for.......",ID_REGION)
                x = out_sel.groupby('time').mean(...)
                tg_id = nuts.NUTS_ID[ID_REGION]
                
                x.tg.to_pandas().to_csv('{}_{}.csv'.format(tg_id, year), header= ['{}'.format(tg_id)])
                df = pd.read_csv("{}_{}.csv".format(tg_id, year))
                #df.to_excel("{}.xlsx".format(tg_id))
                df['time'] = pd.to_datetime(df['time'])
                monthly_min = df.groupby(df.time.dt.month)['{}'.format(tg_id)]
                #----------MAX_TEMP-----------#
                print("\n------MAX TEMP--------")
                max_temp = monthly_min.max()
                max_tempdf = pd.DataFrame(max_temp)
                print(max_tempdf)
                max_tempdf.to_csv('{}_maxtemp_{}.csv'.format(tg_id, year))
                #----------MIN_TEMP-----------#
                print("\n------MIN TEMP--------")
                min_temp = monthly_min.min()
                min_tempdf = pd.DataFrame(min_temp)
                print(min_tempdf)
                min_tempdf.to_csv('{}_mintemp_{}.csv'.format(tg_id, year))
                #----------MEAN_TEMP-----------#
                print("\n------MEAN TEMP--------")
                mean_temp = monthly_min.mean()
                mean_tempdf = pd.DataFrame(mean_temp)
                print(mean_tempdf)
                mean_tempdf.to_csv('{}_meantemp_{}.csv'.format(tg_id, year))
            else:
                pass
        #print(out_sel)
        print("\n------------------------------------------------------------------------------")
        print("\n DONE PROCESSING..")
        print("\n------------------------------------------------------------------------------")
#delete()


In [ ]:
#-----------------------------------------------------------------#
#----------------PROCESS 2019 MIN_TEST TEMPERATURES-------------------#
#-----------------------------------------------------------------#

#The obtained csv files above was copied into different folders. I.e max, mean, and min for each year. Here we 
#use 2019 as an example year
#So this code cds into the dir with the min files and writes the final output there. 
#One needs to install csvkit --- pip install csvkit ---- for this code to work
#We use cskit for file merging. But since it is a command line app, we load it's commands using Sultan 
#library within python. In other words, we are calling bash commands within python
#of course other imported libraries are also installed. 
#The procedure could be inefficient but it works in a clever way.
#ignore the codes below this cell

from sultan.api import Sultan
import pandas as pd
import subprocess
import os

#Function to convert csv_files list to string
def listToString(s):
    # initialize an empty string
    str1 = " "
    # return string 
    return (str1.join(s))

min_path = '/Users/michaelrogo/software/cli_he_code/hacked/processed/temp_2019/min/'
os.chdir(min_path)
pwd = min_path
#print(pwd)
#Run bash code to merge files
with Sultan.load() as sultan:
    items = [f for f in os.listdir(path) if os.path.isfile( os.path.join(path, f))\
             if f.endswith('_2019.csv')]
    items.sort()
    item = items[0]
    rest__of_files = listToString(items[1:])
    sultan.csvjoin('-c time {} {} --left > merged.csv'.format(item, rest__of_files)).run()
    #("csvjoin -c time AL011_mintemp_2019.csv *.csv --left > merged_bash.csv")
print("\nDONE MERGING FILES.........\n")

#Create a final file using pandas

print("CREATING a final file.....")
df = pd.read_csv('merged.csv')
#print(df)
month = df['time'] - 1
q = month // 3 + 1
q = pd.DataFrame(q)
df = df.join(q, lsuffix='_months', rsuffix='_quater')
#print(df.loc[df['time_quater'] == 1])
#print("------------------------------------------\n")
maxTemp = df.loc[df['time_quater'] == 1].min()
results = pd.DataFrame(maxTemp)
results.to_csv('min_mergedTemps.csv')
print("-------DONE PROCESSING----------")

In [ ]:
"""
#Now apply the mask on the gridded dataset d.
#Select only first timestep to speed up the process
#Zoom only on the European continent
print("Masking takes a while, please wait........")
mask = nuts_mask_poly.mask(d.isel(time=0), lat_name='latitude', lon_name='longitude')
#mask
print("Mask done")
"""

In [ ]:
"""
#Plot the figure 
plt.figure(figsize=(12,8))
ax = plt.axes()
mask.plot(ax = ax)
nuts.plot(ax = ax, alpha = 0.8, facecolor = 'none', lw = 1)
"""

### Extract time-series


In [ ]:
"""
#countries = countries[countries['LEVL_CODE'] == querry]
#nuts.NUTS_ID
#print(nuts[nuts['NUTS_ID'] == ID_REGION])

nuts_ids = []
for i in my_dict:
    ID_REGION = i
    #print(ID_REGION)
    nuts_ids.append(ID_REGION)
    #print(nuts_ids)
#print(nuts.NUTS_ID[ID_REGION])

#print(type(my_dict))

"""

In [ ]:
"""nuts_ids = []
for i in my_dict:
    ID_REGION = i
    nuts_ids.append(ID_REGION)
    lat = mask.latitude.values
    lon = mask.longitude.values
    sel_mask = mask.where(mask == ID_REGION).values
    #print(sel_mask)
    a = []
    id_lon = lon[np.where(~np.all(np.isnan(sel_mask), axis=0))]
    id_lat = lat[np.where(~np.all(np.isnan(sel_mask), axis=1))]
    if len(id_lon) | len(id_lat) :
        #print(ID_REGION, "\t lon-->", id_lon)
        #print(ID_REGION, "\t lat-->", id_lat)
        print("\n")
        out_sel = d.sel(latitude = slice(id_lat[0], id_lat[-1]), longitude = slice(id_lon[0], id_lon[-1])).compute().where(mask == ID_REGION)
        #print(out_sel) 
        print("\n------Extracting Temp Data for.......",ID_REGION)
        x = out_sel.groupby('time').mean(...)
        tg_id = nuts.NUTS_ID[ID_REGION]
        x.tg.to_pandas().to_csv('{}_{}.csv'.format(tg_id, year), header= ['{}'.format(tg_id)])
        df = pd.read_csv("{}_{}.csv".format(tg_id, year))
        #df.to_excel("{}.xlsx".format(tg_id))
        df['time'] = pd.to_datetime(df['time'])
        monthly_min = df.groupby(df.time.dt.month)['{}'.format(tg_id)]
        max_temp = monthly_min.max()
        max_tempdf = pd.DataFrame(max_temp)
        print(max_tempdf)
        #max_tempdf.to_csv('{}_maxtemp_{}.csv'.format(tg_id, year))
    else:
        pass
#print(out_sel)
print("\n------------------------------------------------------------------------------")
print("\n DONE PROCESSING..")
print("\n------------------------------------------------------------------------------")
"""